<a href="https://colab.research.google.com/github/ysterin/deep-pointing/blob/master/deep_pointing_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install line-profiler
%load_ext line_profiler
import numpy as np
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch
import tensorflow as tf
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import Module
from torch.autograd import Variable as V
accelerator


    100% |████████████████████████████████| 92kB 7.0MB/s 
  Running setup.py bdist_wheel for line-profiler ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/05/7d/9b/aafbe8d78dc2b2c644d2efd2f060ab3258143860142575193a
Successfully built line-profiler


'cu80'

In [0]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1t4zss4j8GkqkPEKIBXpYD6brm2G3Y8DG'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentFile('wikisource.zip')))

In [0]:
!unzip wikisource.zip
!ls

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving ta1.txt to ta1.txt
User uploaded file "ta1.txt" with length 3062255 bytes


In [5]:
from random import shuffle
import os
from os import path
text = ''
files_list = os.listdir('./wikisource')
shuffle(files_list)
texts = []
for f in files_list:
  with open("./wikisource/"+f, 'rb') as file:
    t = file.read().decode('utf-8')
    texts.append(t)
    if len(t) > 10000:
      text += t
len(text)

46464140

In [5]:
text = open("ta1.txt", 'r').read()
len(text)

1630922

In [6]:
import re
text = re.sub(chr(64288), chr(1506), text)
text = re.sub(chr(64291), chr(1492), text)
chars = list(set(text))
letters, pointings = [chr(i) for i in range(ord('א'), ord('ת') + 1)], [chr(i) for i in list(range(1455, 1468)) + [1479, 1456]]
print( letters, pointings)
text = ''.join([c for c in text if c in letters or c in pointings or c in [' ', '\n', '\t']])

['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'ך', 'כ', 'ל', 'ם', 'מ', 'ן', 'נ', 'ס', 'ע', 'ף', 'פ', 'ץ', 'צ', 'ק', 'ר', 'ש', 'ת'] ['֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ׇ', 'ְ']


In [7]:
import re
chars = sorted(set(text))
pointing_chars = [c for c in chars if 1455<ord(c)<1468 or ord(c) in [1479, 65533]]
non_pointing_chars = [c for c in chars if not (1455<ord(c)<1480 or ord(c) == 65533) ]
pointing_chars, non_pointing_chars
special_chars = ['.', '[',']','}', '{', '(', ')', '\\', '\/']
np_chars_pattern = ['\\'+c for c in non_pointing_chars if c in special_chars] + [c for c in non_pointing_chars if c not in special_chars]
print(np_chars_pattern)
np_pattern = '[' + ''.join(np_chars_pattern) + ']'
np_pattern

unpointed_text = ''.join(re.findall(np_pattern, text))
pointings = re.split(np_pattern, text)[1:]
print(len(unpointed_text), len(pointings))
len(''.join([''.join(z) for z in zip(pointings, unpointed_text)])) == len(text)
_text = ''.join([''.join(z) for z in zip(unpointed_text, pointings)])
for i in range(len(text)):
  if _text[i] != text[i]:
    print(i, text[i], _text[i])
    
ps2ids = {p:i for i, p in enumerate(pointing_chars)}
cs2ids = {c:i for i, c in enumerate(non_pointing_chars)}
ids2ps = {i:p for i, p in enumerate(pointing_chars)}
ids2cs = {i:c for i, c in enumerate(non_pointing_chars)}

from keras.utils import to_categorical
import numpy as np
from keras import utils
vocab_size = len(cs2ids)
X = np.asarray([cs2ids[c] for c in unpointed_text], dtype=np.int64)
#X = utils.to_categorical(X)
N_points = len(pointing_chars)
N = len(unpointed_text)
y = np.zeros(N, dtype=np.int64)
for i in range(N):
  for j in range(N_points):
    if ids2ps[j] in pointings[i]:
      y[i] = j+1
      break
      
print(y[:10])
print(X[:10])

[' ', 'א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'ך', 'כ', 'ל', 'ם', 'מ', 'ן', 'נ', 'ס', 'ע', 'ף', 'פ', 'ץ', 'צ', 'ק', 'ר', 'ש', 'ת']
978882 978882


Using TensorFlow backend.


[0 1 6 0 5 0 0 0 9 9]
[ 0  2 25  1 26 10 27  0  2 25]


In [0]:
from collections import Counter
c = Counter(text)
print (sum([n for p, n in c.items() if p in non_pointing_chars]))
print (sum([n for p, n in c.items()]))
c

In [8]:
from numpy.random import permutation
from keras.utils import Sequence, to_categorical
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='float32')[y]
def max_(arr):
  if isinstance(arr, torch._TensorBase):
    return torch.max(arr)
  return np.max(arr)
def concat(*arrs):
  if isinstance(arrs[0], (torch.autograd.Variable, torch._TensorBase)):
    return torch.stack(arrs)
  return np.asarray(arrs)
def concat1(arrs):
  if isinstance(arrs[0], (torch.autograd.Variable, torch._TensorBase)):
    return torch.stack(arrs)
  return np.asarray(arrs)
print(max_(torch.from_numpy(np.array([[1,2,3],[4,5,6]]))))
class batchSeq(Sequence):
    def __init__(self, X, y, length, stride, batch_size, randlen=True, shuffle = True, torch=False, x_categorical=False, y_categorical=False):
      self._N = X.shape[0]
      self.X = X
      self.y = y
      self.length = length
      self.stride = stride
      self.bs = batch_size
      self.index = 0
      self.randlen = randlen
      self.shuffle = shuffle
      self.torch = torch
      self.x_categorical = x_categorical
      self.y_categorical = y_categorical
      self.vocab_size = max_(X) + 1
      self.y_dim = max_(y) + 1
    
    def __len__(self):
      return ((self._N-self.length)//self.stride)//self.bs

    def __getitem__(self, idx):
      length = self.length

      if self.randlen:
        length = np.random.randint(length-10, length+10)
        if np.random.rand(1)[0] < 0.05:
          length = length//2
      bx = concat1([self.X[self.stride*(self.bs*idx+i):self.stride*(self.bs*idx+i)+length] for i in range(self.bs)])
      by = concat1([self.y[self.stride*(self.bs*idx+i):self.stride*(self.bs*idx+i)+length] for i in range(self.bs)])
      if self.x_categorical:
        bx = to_categorical(bx, self.vocab_size)
      if self.y_categorical:
        by = to_categorical(by, self.y_dim)
      if self.torch:
        bx = V(torch.from_numpy(bx).cuda())
        by = V(torch.from_numpy(by).cuda())
      return bx, by
    
    def __iter__(self):
      self.index = 0
      if self.shuffle: self.permute = permutation(self.__len__())
      return self
    
    def __next__(self):
      if self.index < self.__len__():
        self.index += 1
        if self.shuffle: return self.__getitem__(self.permute[self.index-1])
        else: return self[self.index-1]
      else:
        raise StopIteration

6


In [0]:
X = X.astype(np.int64)
y = y.astype(np.int64)
X = torch.from_numpy(X).cuda()
y = torch.from_numpy(y).cuda()
val_id = int(X.shape[0]*0.1)
test_id = int(X.shape[0]*0.8)
X_val, X_trn, X_test = X[:val_id], X[val_id:test_id], X[test_id:]
y_val, y_trn, y_test = y[:val_id], y[val_id:test_id], y[test_id:]

In [35]:
val_id, test_id

(2696966, 21575735)

In [0]:
class MyRNN(Module):
  def __init__(self, inpsize, hidsize, activation='tanh'):
    super().__init__()
    self.hidsize = hidsize
    self.w_ih = nn.Parameter(torch.zeros((hidsize, inpsize)).cuda())
    torch.nn.init.xavier_normal(self.w_ih)
    self.w_hh = nn.Parameter(torch.eye(hidsize).cuda())
    self.bias = nn.Parameter(torch.zeros(hidsize).cuda())
    if activation=='tanh':
      self.s = F.tanh
    elif activation=='relu':
      self.f = F.relu
  def forward(self, x, h=None):
    if h is None:
      h = V(torch.zeros(self.hidsize).cuda(), requires_grad=False)
    return self.s(F.linear(x, self.w_ih, self.bias) + F.linear(h, self.w_hh))
# my = MyRNN(8, 12)
# list(my.parameters())
# x = V(torch.randn((16, inps)).cuda())
# h0 = V(torch.randn((16, hids)).cuda())
# my(x)

In [0]:
import pdb
class RecurrentLayer(Module):
  def __init__(self, cell, return_sequence=True, dropout=0.0):
    super().__init__()
    self.ret_seq = return_sequence
    self.cell = cell
    self.dropout = dropout

  def forward(self, x, h0=None):
    bs, bptt, inpdim = x.size()
    hidden = h0
    hiddens = []
    for i in range(bptt):
      hidden = self.cell(x[:,i,:], hidden)  
      hiddens.append(hidden)
    if isinstance(hidden, tuple):
      ys = [h[0] for h in hiddens]
      y = hidden[0]
    else:
      ys = hiddens
      y = hidden
    if self.ret_seq: return torch.stack(ys, 1)
    else:            return y

# inps = 8
# hids = 12
# l = RecurrentLayer(MyRNN(inps, hids))
# x = V(torch.randn((16, 20, inps)).cuda())
# h0 = V(torch.randn((16, hids)).cuda())
# l(x)

In [0]:
import torch
import pdb
from torch import nn
from torch.nn import Module
from torch.distributions import Bernoulli
from scipy.stats import bernoulli
import torch.nn.functional as F
from torch.autograd import Variable as V
def dropout(tensor, mask, p, training):
  if training:
    return tensor*mask
  else:
    return tensor*(1-p)
def sample_bernoulli(dropout, shape):
  p = 1 - dropout
  data = bernoulli.rvs(p, size=shape).astype(np.float32)/p
  return V(torch.from_numpy(data).cuda(), requires_grad=False)
  
class BidirectionalLSTMlayer(Module):
  def __init__(self, inpsize, hidsize, in_dropout=0.0, rec_dropout=0.0, out_dropout=0.0):
    super().__init__()
    self.inpsize = inpsize
    self.hidsize = hidsize
    self.cell_fw = nn.LSTMCell(inpsize, hidsize)
    self.cell_bw = nn.LSTMCell(inpsize, hidsize)
    self.in_dpout, self.rec_dpout, self.out_dpout = in_dropout, rec_dropout, out_dropout
#     self.in_dpout_dist = Bernoulli(torch.ones(self.inpsize)*(1-self.in_dpout))
#     self.rec_dpout_dist = Bernoulli(torch.ones(self.hidsize)*(1-self.rec_dpout))
#     self.out_dpout_dist = Bernoulli(torch.ones(self.hidsize)*(1-self.out_dpout))

    
  def forward(self, x):
    bs, bptt, indim = x.size()
    hfw = V(torch.zeros((bs, self.hidsize)).cuda(), requires_grad=True)
    cfw = V(torch.zeros((bs, self.hidsize)).cuda(), requires_grad=True)
    hbw = V(torch.zeros((bs, self.hidsize)).cuda(), requires_grad=True)
    cbw = V(torch.zeros((bs, self.hidsize)).cuda(), requires_grad=True)    
    hmaskf = sample_bernoulli(self.rec_dpout, (bs, self.hidsize))
    xmaskf = sample_bernoulli(self.in_dpout, (bs, self.inpsize))
    outmaskf = sample_bernoulli(self.out_dpout, (bs, self.hidsize))
    hmaskb = sample_bernoulli(self.rec_dpout, (bs, self.hidsize))
    xmaskb = sample_bernoulli(self.in_dpout, (bs, self.inpsize))
    outmaskb = sample_bernoulli(self.out_dpout, (bs, self.hidsize))
    hs_fwd, hs_bwd = [], []
    
    for i in range(bptt):
      xtf = x[:,i,:]
      xtb = x[:,-i-1,:]
      if self.training:
        hfw = hfw*hmaskf
        hbw = hbw*hmaskb
        xtf = xtf*xmaskf
        xtb = xtb*xmaskb
      hfw, cfw = self.cell_fw(xtf, (hfw, cfw))
      if self.training: hs_fwd.append(hfw*outmaskf)
      else:             hs_fwd.append(hfw)
      hbw, cbw = self.cell_bw(xtb, (hbw, cbw))
      if self.training: hs_bwd.append(hbw*outmaskb)
      else:             hs_bwd.append(hbw)
    ys = [torch.cat((hfw, hbw), -1) for hfw, hbw in zip(hs_fwd, hs_bwd)]
    return torch.stack(ys, 1)
inps = 8
hids = 12
hfw = V(torch.zeros((16, hids)).cuda(), requires_grad=False)
cfw = V(torch.zeros((16, hids)).cuda(), requires_grad=False)
x = V(torch.randn((16, 20, inps)).cuda())
l = BidirectionalLSTMlayer(inps, hids, in_dropout=0.3).cuda()
#n = nn.GRUCell(inps, hids).cuda()
#pdb.set_trace()
# l(x)

In [35]:
sample_bernoulli(0.5, (4, 5))

Variable containing:
 0  2  2  0  2
 2  2  2  2  0
 2  0  0  0  2
 0  0  0  2  2
[torch.cuda.FloatTensor of size 4x5 (GPU 0)]

In [0]:
class BidirectionalGRUlayer(Module):
  def __init__(self, inpsize, hidsize, in_dropout=0.0, rec_dropout=0.0, out_propout=0.0):
    super().__init__()
    self.inpsize = inpsize
    self.hidsize = hidsize
    self.cell_fw = nn.GRUCell(inpsize, hidsize)
    self.cell_bw = nn.GRUCell(inpsize, hidsize)
    self.in_dpout, self.rec_dpout, self.out_dpout = in_dropout, rec_dropout, out_dropout
    
  def forward(self, x):
    bs, bptt, _ = x.size()
    hfw = V(torch.zeros((bs, self.hidsize)).cuda(), requires_grad=True)
    hbw = V(torch.zeros((bs, self.hidsize)).cuda(), requires_grad=True)
    hs_fwd, hs_bwd = [], []
    
    for i in range(bptt):
      hfw = self.cell_fw(x[:,i,:], hfw)
      hs_fwd.append(hfw)
      hbw, cbw = self.cell_bw(x[:,-i-1,:], hbw)
      hs_bwd.insert(0, hbw)
    ys = [torch.cat((hfw, hbw), -1) for hfw, hbw in zip(hs_fwd, hs_bwd)]
    return torch.stack(ys, 1)

In [0]:
%debug

In [0]:
class BidirectionalLayer(Module):
  def __init__(self, cell_fw, cell_bw, merge='concat', dropout=0.0):
    super().__init__()
    self.cell_fw, self.cell_bw = cell_fw, cell_bw
    self.merge = merge
    self.dropout = dropout
    
  def forward(self, x, hfw0=None, hbw0=None):
    bs, bptt, inpdim = x.size()
    hid_fw, hid_bw = hfw0, hbw0
    hids_fw, hids_bw = [], []
    for i in range(bptt):
      hid_fw = self.cell_fw(x[:,i,:], hid_fw)
      if isinstance(hid_fw, tuple): hids_fw.append(hid_fw[0])
      else:                         hids_fw.append(hid_fw)
      hid_bw = self.cell_bw(x[:,-i-1,:], hid_bw)
      if isinstance(hid_fw, tuple): hids_bw.append(hid_bw[0])
      else:                         hids_bw.append(hid_bw)
    if self.merge=='concat':
      ys = [torch.cat((hfw, hbw), -1) for hfw, hbw in zip(hids_fw, hids_bw)]
    return torch.stack(ys, 1)
  
# inps = 8
# hids = 12
# l = BidirectionalLayer(MyRNN(inps, hids), MyRNN(inps, hids))
# x = V(torch.randn((16, 20, inps)).cuda())
# h0 = V(torch.randn((16, hids)).cuda())
# c0 = V(torch.randn((16, hids)).cuda())
# l(x)
# #[n for n, p in l.named_parameters()]

In [122]:
%debug

> /content/mtrand.pyx(993)mtrand.RandomState.randint()

ipdb> exit


In [0]:
def myBdLSTM(inpsize, outsize):
  cell_fw = nn.LSTMCell(inpsize, outsize)
  cell_bw = nn.LSTMCell(inpsize, outsize)
  return BidirectionalLayer(cell_fw, cell_bw)
def myBdRNN(inpsize, outsize):
  cell_fw = MyRNN(inpsize, outsize)
  cell_bw = MyRNN(inpsize, outsize)
  return BidirectionalLayer(cell_fw, cell_bw)
def myBDGRU(inpsize, outsize):
  cell_fw = nn.GRUCell(inpsize, outsize)
  cell_bw = nn.GRUCell(inpsize, outsize)
  return BidirectionalLayer(cell_fw, cell_bw)
class charModelBi(Module):
  def __init__(self, vocab_size, nfac, nhidden, num_layers, output_dim, res = True, dropout = False):
    super().__init__()
    self.nhidden = nhidden
    self.num_layers = num_layers
    self.emb = nn.Embedding(vocab_size, nfac)
    inp_sizes = [nfac] + [nhidden*2]*(num_layers-1)
    hid_sizes = [nhidden]*num_layers
    if dropout:
      self.rnns = nn.ModuleList([BidirectionalLSTMlayer(insz, hidsz, rec_dropout=0.4, in_dropout=0.04, out_dropout=0.4) for insz, hidsz in zip(inp_sizes, hid_sizes)])
    else:
      self.rnns = nn.ModuleList([nn.LSTM(insz, hidsz, 1, batch_first=True, bidirectional=True) for insz, hidsz in zip(inp_sizes, hid_sizes)])
    #self.norms = nn.ModuleList([nn.InstanceNorm1d(hidsz) for hidsz in hid_sizes])
    self.out = nn.Linear(nhidden*2, output_dim)
    self.res = res
  
  def forward(self, x):
    bs = x.size(0)
    bptt = x.size(1)
    h0 = F.Variable(torch.zeros((bs, self.nhidden)).cuda())
    c0 = F.Variable(torch.zeros((bs, self.nhidden)).cuda())
    x = self.emb(x)
    xs = []
    for i, rnn in enumerate(self.rnns):
      out = rnn(x)
      if isinstance(out, tuple):
        out = out[0]
      if i==0 or not self.res: x = out
      else:                    x = x + out

    return F.log_softmax(self.out(x), dim=-1)

In [0]:
def evaluate(model, seq, n=0):
  model.eval()
  acc_loss, acc_acc = 0, 0
  i = 0
  for bx, by in seq:
    i += 1
    
    #pdb.set_trace()
    outputs = model(V(bx))
    target = V(by)
    outputs, target = outputs[:, n:-n-1].contiguous(), target[:, n:-n-1].contiguous()
    loss = nll_loss_seq(outputs, target)
    acc_loss += loss.data.cpu().numpy()[0]
    acc = accuracy(outputs, target)
    acc_acc += acc.data.cpu().numpy()[0]
  print("avarage loss: {}".format(acc_loss/i))
  print("avarage accuracy: {}".format(acc_acc/i))

In [0]:
def nll_loss_seq(inp, targ):
  #print(inp.size(), targ.size())
  sl,bs,nh = inp.size()
  #targ = targ.transpose(0,1).contiguous().view(-1)
  targ = targ.view(-1)
  return F.nll_loss(inp.view(-1,nh), targ)

def cross_entropy_seq(inp, targ):
  #print(inp.size(), targ.size())
  sl,bs,nh = inp.size()
  targ = targ.transpose(0,1).contiguous().view(-1)
  return nn.CrossEntropyLoss()(inp.view(-1,nh), targ)

def accuracy(inp, targ):
  _, inp = torch.max(inp, dim=-1)
  return torch.mean(torch.eq(inp, targ).double())

def fit(model, lr, trn_seq, val_seq=None, epochs=1, batches_per_epoch=-1, wd=0.0):
  if batches_per_epoch < 1:
    batches_per_epoch = len(trn_seq)
  print("number of batches: ", len(trn_seq)*epochs)
  model.cuda()
  interval = 200
  opt = optim.Adam(model.parameters(), lr=lr)
  #scheduler = optim.lr_scheduler.StepLR(opt, interval*3, gamma=0.8)
  losses = []
  i = 0
  for ie in range(epochs):
    i = 0
    acc_i = 0
    acc_loss, acc_acc, acc_i = 0, 0, 0
    for bx, by in trn_seq:
      i += 1
      #scheduler.step()
      if i%batches_per_epoch==0:
        if batches_per_epoch < interval:
          print("avarage training loss at epoch     {}: {}".format(ie, acc_loss/acc_i))
          print("avarage training accuracy at epoch {}: {}".format(ie, acc_acc/acc_i))
        if val_seq:
          evaluate(model, val_seq)
      model.train()
      inp = V(bx)
      opt.zero_grad()
      outputs = model(inp)
      target = V(by)
      loss = nll_loss_seq(outputs, target)
      acc = accuracy(outputs, target)
      losses.append(loss.data.cpu().numpy()[0])
      acc_acc += acc.data.cpu().numpy()[0]
      acc_loss += loss.data.cpu().numpy()[0]
      acc_i += 1
      opt.zero_grad()
      loss.backward()
      for name, param in model.named_parameters():
        if name.split('.')[-1].startswith('weight'): 
          param.data.add_(-lr*wd*param.data)
      opt.step()
      if i%interval==0:
        print("avarage loss at i = {}: {}".format(i, acc_loss/interval))
        print("avarage accuracy at i = {}: {}".format(i, acc_acc/interval))
        acc_loss, acc_acc = 0, 0

      
  return losses, [p.data.cpu().numpy() for p in model.parameters()]

In [37]:
def num_params(model):
  return sum([np.prod(p.size()) for p in model.parameters()])
#num_params(charModelBi(vocab_size, 512, 256, 8, N_points + 1))
vocab_size = len(cs2ids)
bptt = 60
output_dim = N_points + 1
model = charModelBi(vocab_size, 128, 1024, 3, output_dim, res=False, dropout=True).cuda()
print(num_params(model))
i = np.random.randint(len(X_trn)-100000)
trn_seq = batchSeq(X_trn, y_trn, bptt, bptt, 64, randlen=True)
print(len(trn_seq))
val_seq = batchSeq(X_val, y_val, bptt, bptt, 128, randlen=False)
#model(V(trn_seq[0][0]))
#torch.save(model.state_dict(), "my model.pt")
%lprun -f fit losses, _ =  fit(model, 5e-4, trn_seq, val_seq=val_seq, epochs=20, wd=0, batches_per_epoch=len(trn_seq))

59848205
178
number of batches:  3560
avarage training loss at epoch     0: 1.072557281302867
avarage training accuracy at epoch 0: 0.6516658744071547
avarage loss: 0.6232630362113317
avarage accuracy: 0.7847921080508474
avarage training loss at epoch     1: 0.5472142113804144
avarage training accuracy at epoch 1: 0.8145216334444574
avarage loss: 0.46787432581186295
avarage accuracy: 0.8439044844632769
avarage training loss at epoch     2: 0.432099685830585
avarage training accuracy at epoch 2: 0.8558117662312184
avarage loss: 0.39659195641676587
avarage accuracy: 0.8664371468926553
avarage training loss at epoch     3: 0.3655796631918115
avarage training accuracy at epoch 3: 0.8779202462110822
avarage loss: 0.35909072309732437
avarage accuracy: 0.881642831920904
avarage training loss at epoch     4: 0.3240314438349783
avarage training accuracy at epoch 4: 0.8917984767711281
avarage loss: 0.3304392124215762
avarage accuracy: 0.8902939618644067
avarage training loss at epoch     5: 0.29

In [38]:
#model = charModelBi(vocab_size, 24, 256, 2, output_dim, res=False).cuda()
#torch.save(model.state_dict(), "my model.pt")
#trn_seq = batchSeq(X_trn, y_trn, bptt, bptt, 32, randlen=True)
print(len(trn_seq))
#val_seq = batchSeq(X_val[:val_id//4], y_val[:val_id//4], bptt, bptt, 32, randlen=False)
#model.load_state_dict(torch.load("my model.pt"))
%lprun -f fit losses, _ =  fit(model, 5e-5, trn_seq, val_seq=val_seq, epochs=10, wd=0, batches_per_epoch=len(trn_seq))

178
number of batches:  1780
avarage training loss at epoch     0: 0.11546000928781126
avarage training accuracy at epoch 0: 0.9599804683756508
avarage loss: 0.27698013186454773
avarage accuracy: 0.9212680967514123
avarage training loss at epoch     1: 0.11264948340627433
avarage training accuracy at epoch 1: 0.9608672931208203
avarage loss: 0.27716362476348877
avarage accuracy: 0.9212018891242938
avarage training loss at epoch     2: 0.10956004689221328
avarage training accuracy at epoch 2: 0.9618551188739634
avarage loss: 0.2791987198094527
avarage accuracy: 0.9216874117231638
avarage training loss at epoch     3: 0.10888363431486706
avarage training accuracy at epoch 3: 0.9620681015954096
avarage loss: 0.2791239048043887
avarage accuracy: 0.9217205155367233
avarage training loss at epoch     4: 0.10782709174742133
avarage training accuracy at epoch 4: 0.962372800012569
avarage loss: 0.2800520285964012
avarage accuracy: 0.922018449858757
avarage training loss at epoch     5: 0.104250

In [0]:
%lprun -f fit losses, _ =  fit(model, 1e-4, trn_seq, val_seq=val_seq, epochs=30, wd=0, batches_per_epoch=len(trn_seq))

number of batches:  5340
avarage training loss at epoch     0: 0.1016475196906739
avarage training accuracy at epoch 0: 0.964546705920185
avarage loss: 0.28640855848789215
avarage accuracy: 0.9223163841807911
avarage training loss at epoch     1: 0.09981147010447615
avarage training accuracy at epoch 1: 0.9653146550165673
avarage loss: 0.2886110097169876
avarage accuracy: 0.922316384180791
avarage training loss at epoch     2: 0.09614831575397718
avarage training accuracy at epoch 2: 0.9665342119809365
avarage loss: 0.2899316648642222
avarage accuracy: 0.9223715572033897
avarage training loss at epoch     3: 0.09511503162609655
avarage training accuracy at epoch 3: 0.9666148571276607
avarage loss: 0.2919873322049777
avarage accuracy: 0.9231329449152543
avarage training loss at epoch     4: 0.09409962838652444
avarage training accuracy at epoch 4: 0.9670817219525788
avarage loss: 0.292464683453242
avarage accuracy: 0.9232322563559321
avarage training loss at epoch     5: 0.0920099312555

In [26]:
trn_seq = batchSeq(X_trn, y_trn, bptt, bptt, 32, randlen=True)
print(len(trn_seq))
val_seq = batchSeq(X_val[:val_id//4], y_val[:val_id//4], bptt, bptt, 32, randlen=False)
#model.load_state_dict(torch.load("my model.pt"))
%lprun -f fit losses, _ =  fit(model, 2e-4, trn_seq, val_seq=val_seq, epochs=1, wd=0, batches_per_epoch=len(trn_seq)//8)

9832
number of batches:  9832
avarage loss at i = 200: 0.0554526672384236
avarage accuracy at i = 200: 0.9825204876844669
avarage loss at i = 400: 0.0510484278074
avarage accuracy at i = 400: 0.9829468391038791
avarage loss at i = 600: 0.0550358807509474
avarage accuracy at i = 600: 0.9820301764585523
avarage loss at i = 800: 0.04965997311286628
avarage accuracy at i = 800: 0.9834794170506824
avarage loss at i = 1000: 0.05592431938741356
avarage accuracy at i = 1000: 0.9816385109785796
avarage loss at i = 1200: 0.0530246452218853
avarage accuracy at i = 1200: 0.9827929116556521
avarage loss: 0.06634443951637996
avarage accuracy: 0.9787645417853742
avarage loss at i = 1400: 0.05049484431190649
avarage accuracy at i = 1400: 0.9833574252577977
avarage loss at i = 1600: 0.05199368970934302
avarage accuracy at i = 1600: 0.9829193972688631
avarage loss at i = 1800: 0.05213765719905496
avarage accuracy at i = 1800: 0.9829211678805376
avarage loss at i = 2000: 0.05605135989375412
avarage accur

In [36]:
trn_seq = batchSeq(X_trn, y_trn, bptt, bptt, 32, randlen=True)
print(len(trn_seq))
val_seq = batchSeq(X_test[:val_id//4], y_test[:val_id//4], bptt, bptt, 32, randlen=False)
#model.load_state_dict(torch.load("my model.pt"))
%lprun -f fit losses, _ =  fit(model, 3e-4, trn_seq, val_seq=val_seq, epochs=1, wd=0, batches_per_epoch=len(trn_seq)//8)

9832
number of batches:  9832
avarage loss at i = 200: 0.041403167634271085
avarage accuracy at i = 200: 0.9861772033763448
avarage loss at i = 400: 0.041576813798783405
avarage accuracy at i = 400: 0.9862693766555322
avarage loss at i = 600: 0.04075142173824133
avarage accuracy at i = 600: 0.9863678831023514
avarage loss at i = 800: 0.042827661731280386
avarage accuracy at i = 800: 0.9854852155247452
avarage loss at i = 1000: 0.04527867020107806
avarage accuracy at i = 1000: 0.9848085640709724
avarage loss at i = 1200: 0.04601411873823963
avarage accuracy at i = 1200: 0.9844833683473566
avarage loss: 0.05052742458338666
avarage accuracy: 0.9841373315949586
avarage loss at i = 1400: 0.04265186666569207
avarage accuracy at i = 1400: 0.9856582533976124
avarage loss at i = 1600: 0.04489049159456045
avarage accuracy at i = 1600: 0.9847137012932233
avarage loss at i = 1800: 0.04260679536171665
avarage accuracy at i = 1800: 0.9855992055715345
avarage loss at i = 2000: 0.0491508737008553
avar

In [41]:
trn_seq = batchSeq(X_trn, y_trn, bptt, bptt, 32, randlen=True)
print(len(trn_seq))
val_seq = batchSeq(X_test[:val_id//4], y_test[:val_id//4], bptt, bptt, 32, randlen=False)
model.load_state_dict(torch.load("my model.pt"))
%lprun -f fit losses, _ =  fit(model, 3e-5, trn_seq, val_seq=val_seq, epochs=1, wd=0.0, batches_per_epoch=len(trn_seq)//8)

9832
number of batches:  9832
avarage loss at i = 200: 0.03928922806400806
avarage accuracy at i = 200: 0.9867964793335835
avarage loss at i = 400: 0.041206079733092336
avarage accuracy at i = 400: 0.9860861114064587
avarage loss at i = 600: 0.03808613482822693
avarage accuracy at i = 600: 0.9870889891464061
avarage loss at i = 800: 0.03799593140021898
avarage accuracy at i = 800: 0.9872769272158424
avarage loss at i = 1000: 0.03704193865181878
avarage accuracy at i = 1000: 0.9878143787700986
avarage loss at i = 1200: 0.0398629853145394
avarage accuracy at i = 1200: 0.9867635236925478
avarage loss: 0.1684224225243718
avarage accuracy: 0.9519049688541212
avarage loss at i = 1400: 0.041359861278906465
avarage accuracy at i = 1400: 0.9860671728659017
avarage loss at i = 1600: 0.04130492753778526
avarage accuracy at i = 1600: 0.9865324915975258
avarage loss at i = 1800: 0.0429946172167547
avarage accuracy at i = 1800: 0.985887600377906
avarage loss at i = 2000: 0.041932297544553875
avarage

In [30]:
def find_lr(model, seq):
  model.cuda()
  lr = 1e-8
  i = 0
  while lr<1e-2:
    lr *= 2
    bx, by = seq[0]
    outputs = model(V(bx))
    opt = optim.RMSprop(model.parameters(), lr)
    target = V(by)
    loss = nll_loss_seq(outputs, target)
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(lr, loss.data.cpu().numpy()[0])
    
output_dim = N_points+1
model = charModelBi(vocab_size, 128, 768, 3, output_dim, res=False, dropout=True).cuda()
bptt = 60

torch.save(model.state_dict(), "my model.pt")
#trn_seq = batchSeq(X_trn, y_trn, bptt, bptt, 64, randlen=True)
find_lr(model, trn_seq)
model.load_state_dict(torch.load("my model.pt"))


2e-08 2.5704327
4e-08 2.5703752
8e-08 2.5706985
1.6e-07 2.5699944
3.2e-07 2.5692663
6.4e-07 2.568564
1.28e-06 2.5660367
2.56e-06 2.5610957
5.12e-06 2.5525289
1.024e-05 2.5341272
2.048e-05 2.4936934
4.096e-05 2.3889506
8.192e-05 1.9278762
0.00016384 6.0058374
0.00032768 2.5563734
0.00065536 3.9217575
0.00131072 3.7425306
0.00262144 4.0890765
0.00524288 2.8857484
0.01048576 19.41905


In [24]:
#torch.save(model.state_dict(), "my model1.pt")
%lprun -f fit losses, _ =  fit(model, 1e-4, trn_seq, val_seq=val_seq, epochs=20, wd=0.0003, batches_per_epoch=len(trn_seq))

number of batches:  6721
avarage loss at i = 200: 0.06504710240289568
avarage accuracy at i = 200: 0.9784239244972671
avarage loss at i = 400: 0.06026203079149127
avarage accuracy at i = 400: 0.9796864708241853
avarage loss at i = 600: 0.06609165752772242
avarage accuracy at i = 600: 0.9782166353861521
avarage loss at i = 800: 0.06241172423120588
avarage accuracy at i = 800: 0.9793155276583703
avarage loss at i = 1000: 0.06496478077955543
avarage accuracy at i = 1000: 0.9781961354726805
avarage loss at i = 1200: 0.06069545143283903
avarage accuracy at i = 1200: 0.9805051591632838
avarage loss at i = 1400: 0.06183864720631391
avarage accuracy at i = 1400: 0.9795414015754207
avarage loss at i = 1600: 0.06440245644189417
avarage accuracy at i = 1600: 0.9789727380983131
avarage training loss at epoch 0: 0.0032342853564868873
avarage training accuracy at epoch 0: 0.04597936797031332
avarage validation loss at epoch 0: 0.06771225292856495
avarage validation accuracy at epoch 0: 0.97808756510

In [35]:
def evaluate(model, seq, n=0):
  model.eval()
  acc_loss, acc_acc = 0, 0
  i = 0
  for bx, by in seq:
    i += 1
    
    #pdb.set_trace()
    outputs = model(V(bx))
    target = V(by)
    outputs, target = outputs[:, n:-n-1].contiguous(), target[:, n:-n-1].contiguous()
    loss = nll_loss_seq(outputs, target)
    acc_loss += loss.data.cpu().numpy()[0]
    acc = accuracy(outputs, target)
    acc_acc += acc.data.cpu().numpy()[0]
  print("avarage loss: {}".format(acc_loss/i))
  print("avarage accuracy: {}".format(acc_acc/i))
  
# test_seq = batchSeq(X_test, y_test, bptt, bptt, 128, randlen=False)
evaluate(model, val_seq, 10)

avarage loss: 0.03316952538024999
avarage accuracy: 0.9900010957703269


In [23]:
test_seq = batchSeq(X_test, y_test, bptt, bptt, 32, randlen=False)
evaluate(model, test_seq, istest=False)

avarage loss: 0.09702100249744172
avarage accuracy: 0.9691199047703793


In [30]:
test_seq = batchSeq(X_test, y_test, bptt, bptt, 32, randlen=False)
evaluate(model, test_seq)

avarage loss: 0.07682160334984087
avarage accuracy: 0.9764578971345108


In [37]:
test_seq = batchSeq(X_test, y_test, bptt, bptt, 32, randlen=False)
evaluate(model, test_seq)

avarage loss: 0.08125722883374044
avarage accuracy: 0.9758737426914711


In [1]:
test_seq = batchSeq(X_test, y_test, bptt, bptt, 32, randlen=False)
evaluate(model, test_seq, 5)

NameError: ignored